In [1]:
import os
import sys
from pathlib import Path
import pickle

import bpy
import open3d as o3d
import numpy as np

sys.path.append("../../")
sys.path.append("../")
from simulation.cloth_3d_util.accessor import Cloth3DCanonicalAccessor
from simulation.pipeline.smpl_simulation_pipeline import smpl_simulation_pipeline
from simulation.cloth_3d_util.util import loadInfo, quads2tris

%load_ext autoreload
%autoreload 2

WARN (bgl): source/blender/python/generic/bgl.c:2654 BPyInit_bgl: 'bgl' imported without an OpenGL backend. Please update your add-ons to use the 'gpu' module. In Blender 4.0 'bgl' will be removed.
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/dcolli23/.local/share/virtualenvs/simulation-7NaVsHdv/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  retur

In [2]:
FILE_ROOT = Path(os.getcwd())
CLOTH3D_PATH = Path(os.path.expanduser("~/DataLocker/datasets/CLOTH3D/training/"))
OUTPUT_ROOT = FILE_ROOT / ".." / "script_output" / "sim_pipeline_driver_test"

# Make the output directory if it doesn't exist.
OUTPUT_ROOT.mkdir(exist_ok=True)

In [3]:
sample_configuration = {
    "sample_id": "00380",
    "garment_name": "Tshirt",
    "grip_vertex_idx": 0
}

sample_configs = [sample_configuration]

In [4]:
accessor = Cloth3DCanonicalAccessor(CLOTH3D_PATH)

In [15]:
for config in sample_configs:
    sample_key = f"{config['sample_id']}_{config['garment_name']}_{config['grip_vertex_idx']}"
    sample_dir = OUTPUT_ROOT / sample_key
    # Make the output sample directory if it doesn't exist.
    sample_dir.mkdir(exist_ok=True)

    result_file = "simulation_result.pk"
    result_path = sample_dir / result_file

    # Get a dictionary containing the data for this sample garment.
    sample_data = accessor.get_sample_data(**config)

    # garment_info = accessor.reader.read_info(config["sample_id"])
    garment_info_mat_filename = CLOTH3D_PATH / config["sample_id"] / "info.mat"
    # print(garment_info_mat_filename)
    garment_info = loadInfo(garment_info_mat_filename)

    if result_path.exists():
        print(f"WARNING! Result file, '{result_path}', already exists! Skipping this run. Reading result instead")
        result_data = pickle.load(result_path.open('rb'))
        continue

    

    args_dict = dict()
    args_dict.update(config)
    args_dict.update(sample_data)
    args_dict["simulation_duration_pair"] = (0, 120)

    result_data = smpl_simulation_pipeline(**args_dict)

    print(f"Writing simulation results to '{result_path}'")
    pickle.dump(result_data, result_path.open('wb'))
    print("Done!")

WARNING! Result file, '/home/dcolli23/code/school/rob599_deeprob/projects/final/garmentnets_tracking/simulation/runners/../script_output/sim_pipeline_driver_test/00380_Tshirt_0/simulation_result.pk', already exists! Skipping this run. Reading result instead


In [16]:
print(result_data.keys())

dict_keys(['cloth_state', 'grip_vertex_idx'])


In [17]:
result_data['cloth_state'].keys()

dict_keys(['verts', 'edges', 'faces', 'uv_verts', 'uv_faces'])

In [18]:
garment_info["outfit"]["Tshirt"].keys()

print(garment_info["outfit"]["Tshirt"]["texture"])
pc_rgb = garment_info["outfit"]["Tshirt"]["texture"]["data"]

{'data': array([0.14440767, 0.86416321, 0.31548023]), 'type': 'color'}


In [19]:
cloth_state = result_data["cloth_state"]
mesh_verts = cloth_state["verts"]
mesh_faces = cloth_state["faces"]
mesh_triangles = quads2tris(mesh_faces)

num_verts = mesh_verts.shape[0]
print("Number of vertices:", num_verts)

# I think the point cloud points are just the vertices but I'm not sure
pc_point = mesh_verts

Number of vertices: 4762


In [22]:


# I think this is just a numpy.arange for the vertices.
# pc_keypoint_idx = point_cloud_group["human_nn_idx"][:].squeeze()
pc_keypoint_idx = np.arange(num_verts)
# keypoint_colors = np.random.random((pc_keypoint_idx.max() + 1, 3))
keypoint_colors = np.random.random((num_verts, 3))

# cloth_vert_human_nn_idx = misc_group['cloth_vert_human_nn_idx'][:]


pc_rgb_color = pc_rgb.astype(np.float32)# / 255
pc_keypoint_color = keypoint_colors[pc_keypoint_idx]

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_point)
# pcd.colors = o3d.utility.Vector3dVector(pc_rgb_color)
# pcd.colors = o3d.utility.Vector3dVector(pc_keypoint_color)

# mesh_verts_shifted = np.array(mesh_verts)
# mesh_verts_shifted[:,0] += 0.5

# mesh = o3d.geometry.TriangleMesh()
# mesh.vertices = o3d.utility.Vector3dVector(mesh_verts_shifted)
# mesh.triangles = o3d.utility.Vector3iVector(mesh_triangles)
# mesh.vertex_colors = o3d.utility.Vector3dVector(keypoint_colors[cloth_vert_human_nn_idx])

# o3d.visualization.draw_geometries([pcd, mesh], mesh_show_wireframe=True, mesh_show_back_face=True)

o3d.visualization.draw_geometries([pcd])